In [ ]:
import csv
import pandas as pd
import os

Find the countries that have both GPI and Gdelt values 


In [ ]:
countries_gpi = []
for i in os.listdir('data/interpolated_gpi/'):
    if i != '.DS_Store':
        countries_gpi.append((i.split('_')[2].split('.')[0]))
        
countries_gdelt = []
for j in os.listdir('data/gdelt_data/'):
    if j != '.DS_Store':
        countries_gdelt.append(j.split('.')[0])
        
all_countries = [country for country in countries_gdelt if country in countries_gpi] 

In [ ]:
len(all_countries)

For every country create a dataframe with all the variables and the GPI values that we will use for machine learning. 

In [ ]:
all_variables_df = {}

for country in all_countries:
    # Read the GPI interpolated data
    gpi_data = pd.read_csv('data/interpolated_gpi/interpolated_gpi_%s.csv' %country)
    gpi_data.astype({'MonthYear': 'str'}).dtypes #convert MonthYear to strings
    all_variables_df[country] = pd.DataFrame({ 'MonthYear': gpi_data.MonthYear, 'GPI': gpi_data.GPI_score})
    all_variables_df[country] = all_variables_df[country].set_index('MonthYear')
    
    #Read the gdelt data as extracted from the Big Query
    country_data = pd.read_csv('data/gdelt_data/%s.csv' %country)
    #Order data based on date, cause after updating they result mixed
    country_data = country_data.groupby('EventBaseCode').apply(pd.DataFrame.sort_values,'MonthYear').reset_index(drop = True)
    # Remove event codes that are not numeric
    country_data.EventBaseCode = pd.to_numeric(country_data.EventBaseCode, errors='coerce')
    country_data = country_data.dropna()
    country_data = country_data.reset_index(drop=True)
    country_data.EventBaseCode =  country_data.EventBaseCode.astype(int) 
    variables = country_data['EventBaseCode'].unique()
    
    for v in variables: #create small dataframes for each variable code and append in a bigger dataframe with all variables per country
        
        single_variable = country_data.loc[country_data.EventBaseCode == v]
        single_variable = single_variable.reset_index(drop = True)
        df_to_print = pd.DataFrame({'MonthYear': single_variable.MonthYear, 'event_count_' + str(v): single_variable.eventcount})
        df_to_print = df_to_print.set_index('MonthYear')
        all_variables_df[country] = pd.concat([all_variables_df[country], df_to_print], axis=1) 

    #For machine learning
    all_variables_df[country] = all_variables_df[country].fillna(0)
    all_variables_df[country].to_csv('data/all_variables_and_GPI_monthly_all_countries/all_variables_%s.csv' %(country))
    
    